# Setup

In [2]:
# Import necessary modules

!pip install nltk

import getpass
import os
from enum import Enum
from operator import itemgetter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#NLTK import
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re


Defaulting to user installation because normal site-packages is not writeable


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Loading / Splitting dataset

In [24]:
# Load and split documents
pdf_directory = "./PDF_FILES"
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=25, add_start_index=True)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, add_start_index=True)

all_docs = []
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        #print(filename)
        pdfloader = PyPDFLoader(os.path.join(pdf_directory, filename))
        docs = pdfloader.load()
        all_docs.extend(docs)

print(len(all_docs))

BIKE SHOPMANUALA GUIDE FOR BA SIC REP AIRS2021-2022
UCSB ASSOCIATEDSTUDENTS

101


# Embedding Phase


In [3]:

# Embedding Phase
sentence_transformer_ef = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(collection_name="full_documents", embedding_function=sentence_transformer_ef)


# Other embedding models are available in:
# - lang_chain_embedding_models
# - sentence-transformers
# - Kaggle (includes all ML models, not just embedding ones)

/home/user/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/user/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-06-19 11:55:58.285554: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operati

# Retrieving Phase

In [7]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

retriever.add_documents(all_docs)
len(list(store.yield_keys()))

sub_docs = vectorstore.similarity_search("bike")
print((sub_docs[0].page_content))
print("========")
retrieved_docs = retriever.invoke("bike")
print((retrieved_docs[0].page_content))
#len(retrieved_docs[0].page_content)

bike. Again be careful not to push the rivetout of thelink!If someone brings a bike in without a chainor you
1.Inspect the chain forrust, broken, twisted or bentlinks. Anoverly rusted chain that cannot bendneedsto be replaced.Achain with "play" in its links is indicative of wearand the chainmay need to be replaced. Use thechain checkertooltomeasure the amount of play in a suspect multi-speedchain.(Ask a fellow mechanic how to use this tool). Usethe chainbreaker tool to break the chain as stated in theSingleSpeedChain Replacement Tutorial. *Do not push the pin allthe wayout.2.Next, get the correct size chain that correspondswith thenumber of speeds on the rear wheel of the bike.3.Again, new chains will likely have excess chains thatneed to beremoved to match the size of the old chain. Lay thenew andold chain next to each other and align each link toidentify thelink where the new chain must be broken. Remove thelinkwith the same technique used to originally removethe chainfrom the bike. 

# Answer generation phase

In [8]:

#get password using GUI
os.environ["OPENAI_API_KEY"] = getpass.getpass()

#class to grade llm answer
class gradeEnum(str, Enum):
    correct = "correct"
    incorrect = "incorrect"

#class to eval answer by question and context
class LLMEvalResult(BaseModel):
    grade: gradeEnum = Field(description="Final grade label. Accepted labels: correct, incorrect")
    description: str = Field(description="Explanation of why the specific grade was assigned. Must be concise. Not more than 2 sentences")
    llm_answer: str = Field(description="The original LLM answer evaluated", default="")
    context: str = Field(description="The context used for the evaluation", default="")

json_parser = JsonOutputParser(pydantic_object=LLMEvalResult)


# LLM for answering questions and evaluating Q-A pairs
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm_selfeval = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Define prompts
qa_prompt = ChatPromptTemplate.from_messages([
    ("human", "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Question: {question} Context: {context} Answer:")
])

qa_eval_prompt_text = """
You are a teacher evaluating a test. 
You are provided with a question along with an answer for the question written by a student. Evaluate the question-answer pair and provide feedback.
{format_instructions}
Question: {question}
Answer: {answer}
"""

qa_eval_prompt = PromptTemplate(
    template=qa_eval_prompt_text,
    input_variables=["question", "answer"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()}
)

qa_eval_prompt_with_context_text = """
You are a teacher evaluating a test. 
You are provided with a question along with an answer for the question written by a student. Evaluate the question-answer pair using the provided context and provide feedback. Only mark the answer as correct if it agrees with the provided context.
{format_instructions}
Context: {context}
Question: {question}
Answer: {answer}
"""

qa_eval_prompt_with_context = PromptTemplate(
    template=qa_eval_prompt_with_context_text,
    input_variables=["question", "answer", "context"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()}
)

# Define RAG Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def retrieve_answer(output):
    return output.content

def process_eval_output(output, llm_answer, context):
    if output['grade'] == "correct":
        output['llm_answer'] = llm_answer
    output['context'] = context
    return output

rag_chain = (
    RunnableParallel(
        context=retriever | format_docs,
        question=RunnablePassthrough()
    ) |
    RunnableParallel(
        answer=qa_prompt | llm | retrieve_answer,
        question=itemgetter("question"),
        context=itemgetter("context")
    ) |
    RunnableParallel(
        eval_result=qa_eval_prompt_with_context | llm_selfeval | json_parser,
        context=itemgetter("context"),
        question=itemgetter("question"),
        answer=itemgetter("answer")
    ) |
    (lambda x: process_eval_output(x['eval_result'], x['answer'], x['context']))
)

# Get user prompt and invoke RAG chain
user_prompt = "what does happen to bracket components over time?"
json_answer = rag_chain.invoke(user_prompt)
print(f"Prompt: {user_prompt}\n\nContext: {json_answer['context']}\n\nAnswer: {json_answer['llm_answer']}\n\nEvaluation: {json_answer['grade']}\n\nAnswer description: {json_answer['description']}")


Prompt: what does happen to bracket components over time?

Context: bracket bearings. During rotation, the bearings roll on the bottombracket cup surfaces. Over time, the bottom bracketsystem canbecome loose and this can accelerate the wear on thebearings andcup surfaces. It is important to properly maintainbottom bracketcomponents to extend their lifetime.Typically, inthe shop weseepress-fittedone-piece bottom brackets(common onbeach cruisers),English threaded three-piece bottom brackets, andEnglishthreadedcartridge bottom brackets.
One-Piece Bottom Brackets
In one-piece bottom brackets, one S-shaped piece ofsteel forms thecrank arms and goes from the pedal, through the bottombracket, tothe other pedal. This crank also acts as the bottombracket bearingaxle. The crank arm is threaded in the middle andacts as the bottombracket’s spindle. The crank drive side threadingis right-handthreaded, and the non-drive side (left side) is left-handthreaded. It isnecessary to first remove the left s